<a href="https://colab.research.google.com/github/hoonzi-s/xgboost/blob/main/04.%20%EA%B7%B8%EB%A0%88%EC%9D%B4%EB%94%94%EC%96%B8%ED%8A%B8%20%EB%B6%80%EC%8A%A4%ED%8C%85%EC%97%90%EC%84%9C%20XGBoost%EA%B9%8C%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
# xgb.set_config(verbosity = 0)

In [4]:
head = 'https://raw.githubusercontent.com/rickiepark/handson-gb/main/Chapter'
tail = '04/bike_rentals_cleaned.csv'
link = head + tail

df_bikes = pd.read_csv(link)
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [6]:
x_bikes = df_bikes.iloc[:, :-1]
y_bikes = df_bikes.iloc[:, -1]
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x_bikes, y_bikes, random_state = 2)

In [7]:
from sklearn.tree import DecisionTreeRegressor
tree_1 = DecisionTreeRegressor(max_depth = 2, random_state = 2)
tree_1.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [8]:
y_train_pred = tree_1.predict(x_train)
y2_train = y_train - y_train_pred

In [11]:
tree_2 = DecisionTreeRegressor(max_depth = 2, random_state = 2)
tree_2.fit(x_train, y2_train)
y2_train_pred = tree_2.predict(x_train)
y3_train = y2_train - y2_train_pred

tree_3 = DecisionTreeRegressor(max_depth = 2, random_state = 2)
tree_3.fit(x_train, y3_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [13]:
y1_pred = tree_1.predict(x_test)
y2_pred = tree_2.predict(x_test)
y3_pred = tree_3.predict(x_test)

y_pred = y1_pred + y2_pred + y3_pred
from sklearn.metrics import mean_squared_error as MSE
MSE(y_test, y_pred) ** 0.5

911.0479538776444

In [18]:
res_1 = y_train - np.mean(y_train)

tree_1.fit(x_train, res_1)
pred_1 = tree_1.predict(x_train)
res_2 = y_train - pred_1

tree_2.fit(x_train, res_2)
pred_2 = tree_2.predict(x_train)
res_3 = y_train - (pred_1 + pred_2)

tree_3.fit(x_train, res_3)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [19]:
pred_all = tree_1.predict(x_test) + tree_2.predict(x_test) + tree_3.predict(x_test)
MSE(y_test, pred_all, squared = False)    # RMSE 값 구하기

911.0479538776432

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

In [21]:
gbr = GradientBoostingRegressor(max_depth = 2, random_state = 2, n_estimators = 3, learning_rate = 1)

In [22]:
gbr.fit(x_train, y_train)
y_pred = gbr.predict(x_test)
MSE(y_test, y_pred, squared = False)

911.0479538776439

In [23]:
gbr = GradientBoostingRegressor(max_depth = 2, random_state = 2, n_estimators = 30, learning_rate = 1)
gbr.fit(x_train, y_train)
y_pred = gbr.predict(x_test)
MSE(y_test, y_pred, squared = False)

857.1072323426944

In [24]:
gbr = GradientBoostingRegressor(max_depth = 2, random_state = 2, n_estimators = 300, learning_rate = 1)
gbr.fit(x_train, y_train)
y_pred = gbr.predict(x_test)
MSE(y_test, y_pred, squared = False)

936.3617413678853

In [25]:
gbr = GradientBoostingRegressor(max_depth = 2, random_state = 2, n_estimators = 300)
gbr.fit(x_train, y_train)
y_pred = gbr.predict(x_test)
MSE(y_test, y_pred, squared = False)

653.7456840231495

In [26]:
lr_values = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0]

In [28]:
for value in lr_values:
  gbr = GradientBoostingRegressor(max_depth = 2, random_state = 2, n_estimators = 300, learning_rate = value)
  gbr.fit(x_train, y_train)
  y_pred = gbr.predict(x_test)
  rmse = MSE(y_test, y_pred, squared = False)
  print(f'학습률: {value}, 점수: {rmse}')

학습률: 0.001, 점수: 1633.0261400367258
학습률: 0.01, 점수: 831.5430182728547
학습률: 0.05, 점수: 685.0192988749717
학습률: 0.1, 점수: 653.7456840231495
학습률: 0.15, 점수: 687.666134269379
학습률: 0.2, 점수: 664.312804425697
학습률: 0.3, 점수: 689.4190385930236
학습률: 0.5, 점수: 693.8856905068778
학습률: 1.0, 점수: 936.3617413678853


In [32]:
depths = [None, 1, 2, 3, 4]
for depth in depths:
  gbr = GradientBoostingRegressor(max_depth = depth, random_state = 2, n_estimators = 300)
  gbr.fit(x_train, y_train)
  y_pred = gbr.predict(x_test)
  rmse = MSE(y_test, y_pred, squared = False)
  print(f'최대 깊이: {depth}, 점수: {rmse}')

최대 깊이: None, 점수: 869.2788645118395
최대 깊이: 1, 점수: 707.8261886858736
최대 깊이: 2, 점수: 653.7456840231495
최대 깊이: 3, 점수: 646.4045923317708
최대 깊이: 4, 점수: 663.048387855927


In [36]:
ㅁ = 123
ㅁ

123

1